<a href="https://colab.research.google.com/github/jakecho1108/Project1/blob/main/Project_1_Part_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Modeling & preprocessing import
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer,make_column_transformer,make_column_selector
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer

In [13]:
df = pd.read_csv('/content/sales_predictions.csv')
df.dropna(axis=1, inplace=True) 
df['Item_Fat_Content']=df['Item_Fat_Content'].replace('LF','Low Fat')
df['Item_Fat_Content']=df['Item_Fat_Content'].replace('low fat','Low Fat')
df['Item_Fat_Content']=df['Item_Fat_Content'].replace('reg','Regular')
df.head()

,Item_Identifier,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Tier 1,Supermarket Type1,3735.1380
1,DRC01,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Tier 3,Supermarket Type2,443.4228
2,FDN15,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Tier 1,Supermarket Type1,2097.2700
3,FDX07,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Tier 3,Grocery Store,732.3800
4,NCD19,Low Fat,0.000000,Household,53.8614,OUT013,1987,Tier 3,Supermarket Type1,994.7052


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Fat_Content           8523 non-null   object 
 2   Item_Visibility            8523 non-null   float64
 3   Item_Type                  8523 non-null   object 
 4   Item_MRP                   8523 non-null   float64
 5   Outlet_Identifier          8523 non-null   object 
 6   Outlet_Establishment_Year  8523 non-null   int64  
 7   Outlet_Location_Type       8523 non-null   object 
 8   Outlet_Type                8523 non-null   object 
 9   Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(3), int64(1), object(6)
memory usage: 666.0+ KB


In [21]:
ItemFat = {'Low Fat':0,
           'Regular' : 1}
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace(ItemFat)
df['Item_Fat_Content'].value_counts()

2    3350
1    2785
0    2388
Name: Outlet_Location_Type, dtype: int64

In [22]:
LocationTypes = {'Tier 1':0,
           'Tier 2' : 1,
           'Tier 3' : 2}
df['Outlet_Location_Type'] = df['Outlet_Location_Type'].replace(LocationTypes)
df['Outlet_Location_Type'].value_counts()

2    3350
1    2785
0    2388
Name: Outlet_Location_Type, dtype: int64

In [24]:
target = 'Item_Outlet_Sales'

X = df.drop(columns=target).copy()
y = df[target].copy()
X.head()

,Item_Identifier,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type
0,FDA15,0,0.016047,Dairy,249.8092,OUT049,1999,0,Supermarket Type1
1,DRC01,1,0.019278,Soft Drinks,48.2692,OUT018,2009,2,Supermarket Type2
2,FDN15,0,0.016760,Meat,141.6180,OUT049,1999,0,Supermarket Type1
3,FDX07,1,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,2,Grocery Store
4,NCD19,0,0.000000,Household,53.8614,OUT013,1987,2,Supermarket Type1


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [26]:
cat_selector = make_column_selector(dtype_include = 'object')
cat_selector(X_train)

['Item_Identifier', 'Item_Type', 'Outlet_Identifier', 'Outlet_Type']

In [27]:
impute_cat = SimpleImputer(strategy='most_frequent')
encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)

cat_pipe = make_pipeline(impute_cat,encoder)
cat_pipe


Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [28]:
cat_pipe.fit_transform(X_train[cat_selector(X_train)])

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [29]:
num_selector = make_column_selector(dtype_include='number')
num_selector(X_train)

['Item_Fat_Content',
 'Item_Visibility',
 'Item_MRP',
 'Outlet_Establishment_Year',
 'Outlet_Location_Type']

In [30]:
scaler = StandardScaler()
scaler.fit_transform(X_train[num_selector(X_train)])

array([[-0.7403206 , -0.71277507,  1.82810922,  1.32784893,  1.08494779],
       [ 1.35076614, -1.29105225,  0.60336888,  1.32784893,  1.08494779],
       [ 1.35076614,  1.81331864,  0.24454056,  0.13618724, -1.38477667],
       ...,
       [-0.7403206 , -0.92052713,  1.52302674,  0.49368575, -0.14991444],
       [-0.7403206 , -0.2277552 , -0.38377708,  1.0895166 , -0.14991444],
       [-0.7403206 , -0.95867683, -0.73836105, -0.10214509, -1.38477667]])

In [31]:
preprocessor = make_column_transformer((cat_pipe,cat_selector),
                                       (scaler,num_selector))
preprocessor

ColumnTransformer(transformers=[('pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False,
                                                                sparse_output=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f5a14e98100>),
                                ('standardscaler', StandardScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f5a176eda60>)])

In [32]:
preprocessor.fit_transform(X_train)


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[ 0.        ,  0.        ,  0.        , ...,  1.82810922,
         1.32784893,  1.08494779],
       [ 0.        ,  0.        ,  0.        , ...,  0.60336888,
         1.32784893,  1.08494779],
       [ 0.        ,  0.        ,  0.        , ...,  0.24454056,
         0.13618724, -1.38477667],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  1.52302674,
         0.49368575, -0.14991444],
       [ 0.        ,  0.        ,  0.        , ..., -0.38377708,
         1.0895166 , -0.14991444],
       [ 0.        ,  0.        ,  0.        , ..., -0.73836105,
        -0.10214509, -1.38477667]])